# Vérification de l’appareil et utilisation de MPS

In [28]:
import torch

# Vérifier si MPS est disponible
if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("Utilisation de l'appareil MPS pour l'entraînement")
else:
    device = torch.device("cpu")
    print("Utilisation de l'appareil CPU pour l'entraînement")

Utilisation de l'appareil MPS pour l'entraînement


# Préparer le jeu de données CIFAR-10

In [29]:
import torchvision
import torchvision.transforms as transforms

# Prétraitement des données : conversion des images en tenseurs et normalisation de chaque canal dans l'intervalle [-1, 1]
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

# Télécharger et charger le jeu de données d'entraînement et de test CIFAR-10
trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=16, shuffle=False, num_workers=2)

# Étiquettes de classification CIFAR-10
classes = ('avion', 'voiture', 'oiseau', 'chat', 'cerf', 'chien', 'grenouille', 'cheval', 'bateau', 'camion')

Files already downloaded and verified
Files already downloaded and verified


# Définir le réseau de neurones convolutif (CNN)

In [30]:
import torch.nn as nn
import torch.nn.functional as F

# Définir un réseau de neurones convolutif
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)  # Première couche de convolution
        self.pool = nn.MaxPool2d(2, 2)   # Couche de pooling
        self.conv2 = nn.Conv2d(6, 16, 5) # Deuxième couche de convolution
        self.fc1 = nn.Linear(16 * 5 * 5, 120)  # Première couche entièrement connectée
        self.fc2 = nn.Linear(120, 84)          # Deuxième couche entièrement connectée
        self.fc3 = nn.Linear(84, 10)           # Dernière couche de sortie avec 10 classes (CIFAR-10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))  # Appliquer la première convolution suivie de ReLU et de pooling
        x = self.pool(F.relu(self.conv2(x)))  # Appliquer la deuxième convolution suivie de ReLU et de pooling
        x = x.view(-1, 16 * 5 * 5)            # Aplatir le tenseur pour les couches entièrement connectées
        x = F.relu(self.fc1(x))               # Appliquer la première couche entièrement connectée suivie de ReLU
        x = F.relu(self.fc2(x))               # Appliquer la deuxième couche entièrement connectée suivie de ReLU
        x = self.fc3(x)                       # Appliquer la couche de sortie
        return x

# Initialiser le modèle et le déplacer sur l'appareil (MPS ou CPU)
net = Net().to(device)

# Entraîner le modèle

In [31]:
import torch.optim as optim

# Définir la fonction de perte et l'optimiseur
criterion = nn.CrossEntropyLoss()  # Entropie croisée
optimizer = optim.SGD(net.parameters(), lr=0.01, momentum=0.9)  # Descente de gradient stochastique

# Entraîner le modèle
for epoch in range(30):  # 10 époques
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        inputs, labels = data
        # Déplacer les données vers l'appareil MPS
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()  # Réinitialiser les gradients

        outputs = net(inputs)  # Passer les données à travers le réseau
        loss = criterion(outputs, labels)  # Calculer la perte
        loss.backward()  # Backpropagation
        optimizer.step()  # Mise à jour des poids

        running_loss += loss.item()
        if i % 200 == 199:  # Afficher la perte toutes les 200 itérations
            print(f'Époque {epoch + 1}, Itération {i + 1} - Perte moyenne : {running_loss / 200:.4f}')
            running_loss = 0.0

# Fonction pour calculer la précision
def calculate_accuracy(loader, model):
    correct = 0
    total = 0
    with torch.no_grad():  # Désactiver la propagation des gradients
        for data in loader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    return 100 * correct / total

# Afficher la précision après chaque époque
train_accuracy = calculate_accuracy(trainloader, net)
test_accuracy = calculate_accuracy(testloader, net)
print(f'Précision entraînement : {train_accuracy:.2f}%, Précision test : {test_accuracy:.2f}%')
print('Entraînement terminé')

# Sauvegarder le modèle
torch.save(net.state_dict(), 'cifar10_net_mps.pth')

Époque 1, Itération 200 - Perte moyenne : 2.2530
Époque 1, Itération 400 - Perte moyenne : 1.9733
Époque 1, Itération 600 - Perte moyenne : 1.7691
Époque 1, Itération 800 - Perte moyenne : 1.6748
Époque 1, Itération 1000 - Perte moyenne : 1.5946
Époque 1, Itération 1200 - Perte moyenne : 1.5542
Époque 1, Itération 1400 - Perte moyenne : 1.4923
Époque 2, Itération 200 - Perte moyenne : 1.4653
Époque 2, Itération 400 - Perte moyenne : 1.4265
Époque 2, Itération 600 - Perte moyenne : 1.3958
Époque 2, Itération 800 - Perte moyenne : 1.3513
Époque 2, Itération 1000 - Perte moyenne : 1.3768
Époque 2, Itération 1200 - Perte moyenne : 1.3451
Époque 2, Itération 1400 - Perte moyenne : 1.3280
Époque 3, Itération 200 - Perte moyenne : 1.2449
Époque 3, Itération 400 - Perte moyenne : 1.2632
Époque 3, Itération 600 - Perte moyenne : 1.2677
Époque 3, Itération 800 - Perte moyenne : 1.2435
Époque 3, Itération 1000 - Perte moyenne : 1.2344
Époque 3, Itération 1200 - Perte moyenne : 1.2535
Époque 3, It

# Évaluer le modèle

In [32]:
correct = 0
total = 0

# Désactiver la propagation en arrière pour l'évaluation
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images, labels = images.to(device), labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)  # Prendre l'indice avec la probabilité la plus élevée
        total += labels.size(0)  # Compter le nombre total d'images
        correct += (predicted == labels).sum().item()  # Compter les prédictions correctes

print(f'Précision du réseau sur les 10000 images de test : {100 * correct / total}%')

Précision du réseau sur les 10000 images de test : 59.03%


# Modèle de CoreML

In [33]:
example_input = torch.rand(1, 3, 32, 32).to(device)
traced_model = torch.jit.trace(net, example_input)

In [34]:
import coremltools as ct

net.eval()

coreml_model = ct.convert(
    traced_model,
    inputs=[ct.ImageType(name="image", shape=(1, 3, 32, 32), scale=1/127.5, bias=[-1, -1, -1])],
    classifier_config=ct.ClassifierConfig(class_labels=list(classes))
    )

coreml_model.save("CIFAR10NetMPS_4.mlpackage")

When both 'convert_to' and 'minimum_deployment_target' not specified, 'convert_to' is set to "mlprogram" and 'minimum_deployment_target' is set to ct.target.iOS15 (which is same as ct.target.macOS12). Note: the model will not run on systems older than iOS15/macOS12/watchOS8/tvOS15. In order to make your model run on older system, please set the 'minimum_deployment_target' to iOS14/iOS13. Details please see the link: https://apple.github.io/coremltools/docs-guides/source/target-conversion-formats.html
Model is not in eval mode. Consider calling '.eval()' on your model prior to conversion
Running MIL backend_mlprogram pipeline: 100%|██████████| 12/12 [00:00<00:00, 2642.64 passes/s]
